In [4]:
import pickle
import numpy as np
from torch.utils.data import Dataset
from PIL import Image

def load_batch(filepath):
    with open(filepath, 'rb') as f:
        batch = pickle.load(f, encoding='bytes')

    # batch[b'data'] → shape (10000, 3072)
    data = batch[b'data'].reshape(-1, 3, 32, 32)
    labels = batch[b'labels']  # list of 10000 labels

    return data, labels

class CIFAR10Pickle(Dataset):
    def __init__(self, files, transform=None):
        self.transform = transform
        self.data = []
        self.labels = []

        for f in files:
            imgs, lbls = load_batch(f)
            self.data.append(imgs)
            self.labels.extend(lbls)

        self.data = np.concatenate(self.data)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img = self.data[idx]
        label = self.labels[idx]

        # convert CHW → HWC for PIL
        img = np.transpose(img, (1, 2, 0))
        img = Image.fromarray(img)

        if self.transform:
            img = self.transform(img)

        return img, label

In [5]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

train_files = [
    "data_batch_1",
    "data_batch_2",
    "data_batch_3",
    "data_batch_4",
    "data_batch_5",
]

test_files = ["test_batch"]

transform = transforms.Compose([
    transforms.ToTensor(),
])

trainset = CIFAR10Pickle(train_files, transform=transform)
testset  = CIFAR10Pickle(test_files, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader  = DataLoader(testset, batch_size=64, shuffle=False)

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 32 -> 16

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 16 -> 8

            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 8 -> 4
        )

        self.fc = nn.Sequential(
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = SimpleCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(5):
    net.train()
    running_loss = 0

    for x, y in trainloader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        out = net(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {running_loss/len(trainloader):.4f}")

Epoch 1 Loss: 1.5362
Epoch 2 Loss: 1.1079
Epoch 3 Loss: 0.9088
Epoch 4 Loss: 0.7849
Epoch 5 Loss: 0.6892


In [8]:
correct = 0
total = 0

net.eval()
with torch.no_grad():
    for x, y in testloader:
        x, y = x.to(device), y.to(device)
        outputs = net(x)
        _, predicted = torch.max(outputs, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

print(f"Accuracy: {100 * correct / total:.2f}%")

Accuracy: 71.80%
